In [13]:
import whisper
from pprint import pprint

In [2]:
# Load the model (you can use "base", "small", "medium", or "large" for better accuracy)
model = whisper.load_model("medium")  

100%|█████████████████████████████████████| 1.42G/1.42G [11:54<00:00, 2.14MiB/s]


In [21]:
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\sat.mp3"
file_path = r"C:\Users\User_1\Desktop\Comm-sounds\1.mp3"
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\2.mp3"
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\3.mp3"
# file_path = r"C:\Users\User_1\Desktop\Comm-sounds\fa_1.m4a"

In [24]:
# Transcribe Persian audio
LANGUAGE = "en"
# LANGUAGE = "fa"

result = model.transcribe(file_path, language=LANGUAGE)

# Print Persian transcription
pprint(result["text"])

(' 300 checks. 300. Dangerous check stabilizers. Sit at 75%. Copy. Standing '
 'by. DOB LPC, turn fine. Declining memory. Turn back for an unexpected A. Up '
 'speed. DOB LPC, up speed successful with no unexpected errors. Copy, DOB. '
 'You are clear for launch. And with that, shut down your visors. O2 on and '
 "prepare for ignition, O2. Copy that and initiating a full 10-4-4. You've got "
 'to check two of your own. LT, L1 clear. Engines on. CP13, sector on. Wing '
 'turning stable. O L H. Woo! Just getting the rock and roll. Over tone stage. '
 'Alright, LL 60. Two minutes 45. Calling two engines manual. Copy, two '
 "engines manual. Turning, we'll be back at 8.2, check. Standing by the full "
 'trails. Overlight deck chairs. On tour manual. 2.4, 2.8G right now and '
 'counting. Access distortion negative one. Standby.')
